# Protein prep:

This uses the script from https://github.com/ljmartin/pdb_to_pdbqt

In short, this downloads the PDB using ProDy, aligns it along the ligand's principal moments of inertia, then runs PDBFixer to add hydrogens and finally minimizes hydrogen coordinates with OpenMM using the AMBER14 forcefield. 

In [1]:
import pdbfixer
import prody
import numpy as np
import py3Dmol
import qrotate



# Download:

In [2]:

fname = prody.fetchPDB('5WIU', folder='./data/')
pdb = prody.parsePDB(fname)


protein = pdb.select('protein and chain A')
print(f'Protein atoms: {protein.numAtoms()}')

#nemonapride has resname AQD 
ligand = pdb.select('resname AQD and chain A')
print(f'Ligand atoms: {ligand.numAtoms()}')


Protein atoms: 2736
Ligand atoms: 27


# Alignt to ligand's PMIs

In [3]:


#get coordinates of protein or ligand:
prot_xyz = protein.getCoords()
lig_xyz = ligand.getCoords()

#centre both on the COM of the ligand:
prot_xyz -= lig_xyz.mean(0)
lig_xyz -= lig_xyz.mean(0)

#rotate both so that ligand lies along it's principal axes:
al = qrotate.Align()
angles = al.align_pcl(lig_xyz, get_angles=True)
#apply the rotation matrices:
prot_xyz = prot_xyz.dot(angles[0]).dot(angles[1])
lig_xyz = lig_xyz.dot(angles[0]).dot(angles[1])

protein.setCoords(prot_xyz)
ligand.setCoords(lig_xyz)

prody.writePDB('./data/AQD_ligand.pdb', ligand, )
prody.writePDB('./data/protein.pdb', protein,)



'./data/protein.pdb'

# Add missing atoms with PDBFixer

In [4]:
from simtk.openmm import app
from simtk import openmm
from simtk import unit
import pdbfixer


fixer = pdbfixer.PDBFixer('./data/protein.pdb')
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)


# Energy-minimize hydrogen coordinates:

In [5]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

system = forcefield.createSystem(fixer.topology, nonbondedMethod=app.CutoffNonPeriodic,
         nonbondedCutoff=0.9*unit.nanometer)


atom_elements = [atom.element.name for atom in fixer.topology.atoms()]
for i in range(system.getNumParticles()):
    if atom_elements[i]!='hydrogen':
        system.setParticleMass( i, 0.0 )

In [6]:
integrator = openmm.LangevinIntegrator(298*unit.kelvin, 1/unit.picosecond, 1*unit.femtosecond)
platform = openmm.Platform.getPlatformByName('CPU')

simulation = app.Simulation(fixer.topology, system, integrator, platform)
simulation.context.setPositions(fixer.positions)
simulation.minimizeEnergy()


# Write output file:

In [7]:
positions = simulation.context.getState(getPositions=True).getPositions()

app.PDBFile.writeFile(fixer.topology, positions, open('./data/proteinH.pdb', 'w'))


# ---- end ----